In [16]:
library(GENIE3)
library(doParallel)
library(dplyr)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
set.seed(27042012) # For reproducibility of results

In [3]:
r=read.delim("Physcomitrella_patens.TAP_annotation.6.tsv",sep="\t",stringsAsFactors=FALSE)

In [4]:
d=read.delim(gzfile("all_genes.vst.tsv.gz"),sep="\t",check.names=FALSE)

In [5]:
d=as.matrix(d)
head(d)

,SRR787291,SRR787292,SRR787293,SRR787294,SRR787295,SRR790623,SRR790624,SRR790625,SRR790254,SRR790259,⋯,65-D14-3,66-LO14,67-LO14-2,68-LO14-3,69-LG14,70-LG14-2,71-LG14-3,72-1-14,73-1-14-2,74-1-14-3
Pp3c1_100.mRNA,9.862156,9.351662,10.623834,8.916050,9.964518,8.777317,8.434408,8.490788,8.997075,8.532412,⋯,10.256458,8.946873,10.769640,10.293939,9.185842,11.330099,10.749067,8.716056,10.795856,9.818899
Pp3c1_1000.mRNA,7.381567,7.559691,8.580751,7.305779,9.089045,10.578629,10.377033,10.091441,9.826457,9.419218,⋯,8.284943,8.206360,8.019149,8.240060,8.342942,8.548026,8.676593,7.724645,7.632214,7.874430
Pp3c1_10000.mRNA,10.427010,9.913541,9.623924,9.697720,9.450088,10.555744,9.812155,10.504621,9.839770,9.944695,⋯,9.518657,9.931283,9.188891,9.936725,9.590335,9.326010,10.018545,10.072072,9.137332,9.567608
Pp3c1_10010.mRNA,7.404392,7.265683,7.924842,7.061676,8.785212,10.227576,10.298046,10.163573,10.220884,9.600121,⋯,8.642796,8.356125,8.505761,8.541699,8.472045,8.560633,8.811026,8.035877,8.379157,8.341642
Pp3c1_10020.seryl_tRNA,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,⋯,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570
Pp3c1_10040.mRNA,8.425789,8.747516,9.015093,8.642327,9.115491,9.740131,9.597228,9.368856,9.346674,9.067421,⋯,8.270825,8.517641,8.403065,8.371686,8.701931,8.555978,8.605236,8.127675,8.066973,8.026048


In [6]:
miRNAs=row.names(d[grep("miRNA",row.names(d)),])

In [128]:
length(miRNAs)

[1] 275

In [7]:
head(r)
nrow(r)

superfamily,locus,inferred_by_rule,TAP_class
BSD domain containing,Pp3c11_12590V3,True,PT
BSD domain containing,Pp3c1_21290V3,True,PT
BSD domain containing,Pp3c20_12230V3,True,PT
BSD domain containing,Pp3c23_10600V3,True,PT
BSD domain containing,Pp3c24_7080V3,True,PT
BSD domain containing,Pp3c26_14480V3,True,PT


[1] 1480

In [127]:
table(r$TAP_class)


 PT  TF  TR 
 79 989 412 

In [99]:
#including TAPs, DEK1, pre-miRNAs and RSL-1,-2 into the regulators list
reg=as.character(c("Pp3c1_38880.mRNA","Pp3c2_8880.mRNA","Pp3c17_17550.mRNA",gsub("V3","",paste(r$locus,"mRNA",sep=".")),miRNAs))
reg2=reg
reg=which(row.names(d)%in% reg)
nrow(d[reg,])

[1] 1737

In [101]:
write.table(data.frame(reg2,TRUE),file="used_regulators.txt",sep="\t",row.names=FALSE,col.names=FALSE,quote=FALSE)

In [9]:
head(d)

,SRR787291,SRR787292,SRR787293,SRR787294,SRR787295,SRR790623,SRR790624,SRR790625,SRR790254,SRR790259,⋯,65-D14-3,66-LO14,67-LO14-2,68-LO14-3,69-LG14,70-LG14-2,71-LG14-3,72-1-14,73-1-14-2,74-1-14-3
Pp3c1_100.mRNA,9.862156,9.351662,10.623834,8.916050,9.964518,8.777317,8.434408,8.490788,8.997075,8.532412,⋯,10.256458,8.946873,10.769640,10.293939,9.185842,11.330099,10.749067,8.716056,10.795856,9.818899
Pp3c1_1000.mRNA,7.381567,7.559691,8.580751,7.305779,9.089045,10.578629,10.377033,10.091441,9.826457,9.419218,⋯,8.284943,8.206360,8.019149,8.240060,8.342942,8.548026,8.676593,7.724645,7.632214,7.874430
Pp3c1_10000.mRNA,10.427010,9.913541,9.623924,9.697720,9.450088,10.555744,9.812155,10.504621,9.839770,9.944695,⋯,9.518657,9.931283,9.188891,9.936725,9.590335,9.326010,10.018545,10.072072,9.137332,9.567608
Pp3c1_10010.mRNA,7.404392,7.265683,7.924842,7.061676,8.785212,10.227576,10.298046,10.163573,10.220884,9.600121,⋯,8.642796,8.356125,8.505761,8.541699,8.472045,8.560633,8.811026,8.035877,8.379157,8.341642
Pp3c1_10020.seryl_tRNA,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,⋯,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570
Pp3c1_10040.mRNA,8.425789,8.747516,9.015093,8.642327,9.115491,9.740131,9.597228,9.368856,9.346674,9.067421,⋯,8.270825,8.517641,8.403065,8.371686,8.701931,8.555978,8.605236,8.127675,8.066973,8.026048


In [10]:
w=GENIE3(exprMatrix=d,nCores=50,verbose=TRUE,regulators=reg)

Tree method: RF
K: sqrt
Number of trees: 1000

Using 50 cores.


In [11]:
w[1:10,1:10]

,Pp3c1_100.mRNA,Pp3c1_1000.mRNA,Pp3c1_10000.mRNA,Pp3c1_10010.mRNA,Pp3c1_10020.seryl_tRNA,Pp3c1_10040.mRNA,Pp3c1_10050.mRNA,Pp3c1_10060.mRNA,Pp3c1_10080.mRNA,Pp3c1_10090.mRNA
Pp3c1_10451.pre_miRNA,1.160744e-06,9.794020e-09,6.288597e-07,3.748409e-08,0.0000000000,7.961691e-07,8.058481e-06,3.102371e-07,5.167709e-06,7.329984e-08
Pp3c1_110.mRNA,4.226683e-05,2.791256e-05,1.181047e-05,2.289612e-05,0.0000000000,6.752086e-05,2.217826e-04,9.657315e-06,2.880088e-05,4.819966e-05
Pp3c1_11190.mRNA,8.410966e-04,7.843237e-05,2.494309e-04,5.000122e-04,0.0058056819,1.980049e-03,6.362608e-04,2.987119e-04,7.788026e-04,4.137744e-03
Pp3c1_1170.mRNA,5.811422e-04,1.249365e-04,2.089998e-04,1.808575e-04,0.0007504103,4.519467e-04,3.934056e-04,3.576981e-04,1.335408e-04,1.966277e-04
Pp3c1_12170.mRNA,4.157131e-04,3.304025e-04,5.190191e-05,7.847912e-05,0.0004538718,3.406858e-04,4.053941e-04,7.470341e-04,1.113713e-04,2.458143e-04
Pp3c1_12290.mRNA,2.205475e-03,1.381285e-04,3.072558e-04,1.510940e-04,0.0010852886,1.146853e-04,7.674232e-04,4.743622e-04,3.408210e-04,9.395023e-05
Pp3c1_12940.mRNA,2.260341e-04,1.720154e-03,4.692471e-04,1.369468e-02,0.0008687729,1.588603e-03,1.400115e-03,4.671864e-04,5.434778e-04,2.239660e-04
Pp3c1_1300.mRNA,1.159639e-04,5.027774e-05,1.041266e-04,5.465288e-05,0.0002462424,3.315996e-04,5.690896e-04,1.453406e-04,1.051012e-04,6.567446e-05
Pp3c1_13300.mRNA,1.358393e-04,1.568179e-04,1.160858e-04,5.460691e-05,0.0023514843,4.278734e-04,2.926677e-04,7.130840e-05,1.448373e-04,1.231120e-04
Pp3c1_13308.mRNA,6.530715e-05,6.923886e-05,1.127541e-04,4.117252e-04,0.0005708430,1.917492e-04,3.194136e-03,7.340051e-04,4.669851e-04,1.457167e-04


In [12]:
links=getLinkList(w)

In [13]:
nrow(links)
head(links)

[1] 61897747

regulatoryGene,targetGene,weight
Pp3c17_930.mRNA,Pp3s340_10.mRNA,0.06778052
Pp3c1_27430.pre_miRNA,Pp3c15_12100.mRNA,0.06724901
Pp3c17_17550.mRNA,Pp3c23_130.mRNA,0.06646932
Pp3c17_18920.pre_miRNA,Pp3s32_420.mRNA,0.06632740
Pp3c15_549.mRNA,Pp3c4_24040.mRNA,0.06400505
Pp3c4_3072.pre_miRNA,Pp3c22_1240.mRNA,0.06353888


In [18]:
ran=links %>%
    group_by(regulatoryGene) %>%
    mutate(regulator_rank = order(order(weight, decreasing=TRUE)))
head(ran)

regulatoryGene,targetGene,weight,regulator_rank
Pp3c17_930.mRNA,Pp3s340_10.mRNA,0.06778052,1
Pp3c1_27430.pre_miRNA,Pp3c15_12100.mRNA,0.06724901,1
Pp3c17_17550.mRNA,Pp3c23_130.mRNA,0.06646932,1
Pp3c17_18920.pre_miRNA,Pp3s32_420.mRNA,0.06632740,1
Pp3c15_549.mRNA,Pp3c4_24040.mRNA,0.06400505,1
Pp3c4_3072.pre_miRNA,Pp3c22_1240.mRNA,0.06353888,1


In [20]:
ran2=ran %>%
    group_by(targetGene) %>%
    mutate(target_rank = order(order(weight, decreasing=TRUE)))
head(ran2)

regulatoryGene,targetGene,weight,regulator_rank,target_rank
Pp3c17_930.mRNA,Pp3s340_10.mRNA,0.06778052,1,1
Pp3c1_27430.pre_miRNA,Pp3c15_12100.mRNA,0.06724901,1,1
Pp3c17_17550.mRNA,Pp3c23_130.mRNA,0.06646932,1,1
Pp3c17_18920.pre_miRNA,Pp3s32_420.mRNA,0.06632740,1,1
Pp3c15_549.mRNA,Pp3c4_24040.mRNA,0.06400505,1,1
Pp3c4_3072.pre_miRNA,Pp3c22_1240.mRNA,0.06353888,1,1


In [21]:
ran2$mutual_rank=sqrt(ran2$regulator_rank*ran2$target_rank)

In [22]:
write.table(ran2,gzfile("GENIE3.all_ranks.ranked_regulator_target_interactions.tsv.gz"),sep="\t",row.names=FALSE,col.names=TRUE,quote=FALSE)